In [57]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import time
import pandas as pd

In [276]:
# Place to replace url for different searches 
url = 'https://www.indeed.com/jobs?q=business+%2445%2C000&l=Miami%2C+FL&start=80&vjk=4a2e69ff71d72d06'

In [277]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException

# Set up Selenium WebDriver
chrome_options = Options()
driver = webdriver.Chrome(options=chrome_options)

# Starting URL
base_url = url  # Replace with the actual URL
driver.get(base_url)
time.sleep(5)

# Initialize an empty list to store job data
output_data = []

# Function to scrape a single page
def scrape_current_page():
    job_listings = driver.find_elements(By.CLASS_NAME, 'result')
    for job in job_listings:
        try:
            # Extract company name and job location
            company_location = job.find_element(By.CLASS_NAME, 'company_location')
            company_name = company_location.find_element(By.CSS_SELECTOR, 'span[data-testid="company-name"]').text
            job_location = company_location.find_element(By.CSS_SELECTOR, 'div[data-testid="text-location"]').text
            
            # Extract job title
            job_title_element = job.find_element(By.CLASS_NAME, 'jobTitle')
            job_title = job_title_element.text
            
            # Extract salary information (if available)
            try:
                salary_element = job.find_element(By.CLASS_NAME, 'salary-snippet-container')
                salary_info = salary_element.find_element(By.CSS_SELECTOR, 'div[data-testid="attribute_snippet_testid"]').text
            except NoSuchElementException:
                salary_info = "None provided"  # Default to "None provided" if salary information is not available
            
            # Append the data to the list
            output_data.append({
                'Company Name': company_name,
                'Job Location': job_location,
                'Job Title': job_title,
                'Salary Information': salary_info
            })
        except NoSuchElementException as e:
            print(f"An error occurred while processing a job listing: {e}")

# Scrape all pages
while True:
    print("Scraping current page...")
    scrape_current_page()
    
    try:
        # Locate the "Next" button based on `data-testid` and click it
        next_button = driver.find_element(By.CSS_SELECTOR, 'a[data-testid^="pagination-page-"]')
        next_page_href = next_button.get_attribute("href")
        driver.get(next_page_href)  # Navigate to the next page
        time.sleep(5)  # Wait for the next page to load
    except NoSuchElementException:
        print("No more pages to scrape.")
        break

# Create a DataFrame from the collected data
job_data_df = pd.DataFrame(output_data, columns=[
    'Company Name', 
    'Job Location', 
    'Job Title', 
    'Salary Information'
])

# Display the DataFrame
print(job_data_df)

# Save the DataFrame to a CSV file
csv_file_name = "job_listings.csv"
job_data_df.to_csv(csv_file_name, mode='a', index=False, header=False)
print(f"Data has been saved to {csv_file_name}")

# Close the WebDriver
driver.quit()


Scraping current page...
Scraping current page...
Scraping current page...
No more pages to scrape.
                             Company Name  \
0                        Landmark Payroll   
1                      MARVIN BOCHNER INC   
2   Right Path Behavioral Health Services   
3                 Wright Consulting Miami   
4                          Pollo Tropical   
5                    Westcliff University   
6                                Swrg Inc   
7                             SmartBarrel   
8                  Williams & Daniel, LLC   
9                      Trident Management   
10                            Bracco N.A.   
11                    Business Sales Firm   
12          City National Bank of Florida   
13                               aiTWorks   
14            Marriott International, Inc   
15                                   Visa   
16                      Quirch Foods, LLC   
17                              GalaxyCon   
18                                 Cordis   
